# College/Trade School Locations vs Population Density by County

### Data
* To access College/Trade School Locations data as a CSV go [here](https://catalog.data.gov/dataset/postsecondary-school-locations-current). This data is for the current year (2021). To access previous years data go [here](https://catalog.data.gov/dataset?q=Postsecondary+School+Locations&sort=views_recent+desc&as_sfid=AAAAAAX9ZMkaz7eiQfOekNnbhdEYJ_8itjie6jAn8b2wxPtQNnjA1ZHKpgjP79FznBsyOcfT4vEF0CTn_bwNzuzZdmMbiIMGPh8wZhAH0Va2HUsHBVOHADouCXxnhMUClXtNRlk%3D&as_fid=dc61e39c8c5d32aa7c8ccced6c5d89d606bd0f3d&ext_location=Flagstaff%2C+AZ+%2886001%29&ext_bbox=-112.1337%2C34.9758%2C-111.2645%2C36.0102&ext_prev_extent=-114.2578125%2C33.65120829920497%2C-109.1162109375%2C37.23032838760387)
* To access population data by county as a CSV go [here](https://worldpopulationreview.com/us-counties/states/az). This data is for the current year (2021).

### Programs Used
* Creating [Choropleth Maps](https://plotly.com/python/choropleth-maps/) using Plotly to create base map
* Using [ScatterGeo](https://plotly.com/python/reference/scattergeo/) from Plotly to layer maps
* Using urlopen from [urllib.request](https://docs.python.org/3/library/urllib.request.html) to load county fips
* Reading csv files using [pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

### Other
* What are [FIPS](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standards) anyways?

In [ ]:
import csv
import os
import pandas as pd
from urllib.request import urlopen
import json
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
school_file = os.path.join('data', 'Postsecondary_School_Locations_2021.csv')
population_file = os.path.join('data', 'AZ_Population_Density_2021.csv')

In [ ]:
df = pd.read_csv(school_file)
arizona_only = df[df["STATE"] == "AZ"]

In [ ]:
population_df = pd.read_csv(population_file)

In [ ]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [ ]:
# Get the List of county names from the population density data
county_names = []
for name in population_df['CTYNAME']:
    county_names.append(' '.join(name.split()[:-1]))

In [ ]:
# Itterate through to get only the counties in Arizona
features = []
fips_list = []
pop_list = []
for county in range(len(counties['features'])):
    county_name = counties['features'][county]['properties']['NAME']
    if county_name in county_names:
        features.append(counties['features'][county])
        # Get the fips id
        fips_id = counties['features'][county]['id']
        if int(fips_id) > int('04000') and int(fips_id) < int('05000'):
            fips_list.append(fips_id)
            # Get the density
            index = population_df[population_df['CTYNAME']==f'{county_name} County'].index.values[0]
            pop_density = population_df.iloc[index]['popDensity']
            pop_list.append(pop_density)
# Replace the current county features with only the features for Arizona
counties['features'] = features
# Create dataframe of fips and coresponding population density
df = pd.DataFrame({'fips': fips_list, 'pop': pop_list})

In [ ]:
# Plot county population density data in Arizona
fig = px.choropleth(df, geojson=counties, locations='fips', color='pop',
                           color_continuous_scale="Viridis",
                           range_color=(0, 30),
                           labels={'pop':'Population Density',
                                  'fips' : 'FIPS'},
                           title = "Population Density vs College and Trade School Locations"
                          )
# Fit to only the counties that we drew
fig.update_geos(fitbounds="locations", visible=False)


#Add scatter plot of school districts
fig.add_trace(go.Scattergeo(
        lon = arizona_only["X"],
        lat = arizona_only["Y"],
        mode = 'markers',
        hovertext = arizona_only["NAME"],
        marker_color = 'rgb(0, 0, 0)'
        ))
fig.write_image("pop_density_vs_higher_education.jpeg") 
fig.show()